In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from plato.planets.metrics import PlanetPopulationMetrics
from plato.planets.populations import PopulationModel
from plato.stars import filter_p1_targets, filter_valid_targets
from plato.visualisation import (
    FigureProcessor,
    contour_plot,
    get_earth_marker,
    get_palette,
    set_plot_defaults,
)

In [3]:
# set_plot_defaults()

figure_directory = f"../figures/06_planet_populations/"
save = True

In [4]:
LOPS2 = pd.read_csv(f"../data/processed/LOPS2_targets.csv")
LOPN1 = pd.read_csv(f"../data/processed/LOPN1_targets.csv")

fields = pd.concat([LOPS2, LOPN1])
fields = filter_valid_targets(fields)
fields = fields[
    [
        "Radius",
        "Mass",
        "[Fe/H]",
        "u1",
        "u2",
        "gaiaV",
        "n_cameras",
        "Population",
    ]
]

fields["cos_i"] = 0
fields["sigma_star"] = 10e-6
fields = fields.rename(
    columns={
        "Radius": "R_star",
        "Mass": "M_star",
        "gaiaV": "Magnitude_V",
    }
)

p1_sample = filter_p1_targets(fields)

## Recreate NGPPS II Table 7

In [5]:
population_model = PopulationModel(
    fields,
    num_embryos=100,
    additional_columns=["Population"],
)

metrics = PlanetPopulationMetrics()

In [6]:
mock = population_model.create_mock_population(additional_columns=["Population"])

In [14]:
table = []
for pop in [None, "Thin Disk", "Thick Disk", "Halo"]:
    number_of_systems = (
        len(fields) if pop is None else len(fields[fields["Population"] == pop])
    )

    metric_table = metrics.calculate_metrics(
        mock,
        number_of_systems=number_of_systems,
        population=pop,
    )

    metric_table["Population"] = "All" if pop is None else pop

    # make population primary index
    m = metric_table.set_index("Population", append=True).reorder_levels(
        ["Population", "Planet Category"]
    )

    table.append(m)

table = pd.concat(table).reindex(
    index=pd.MultiIndex.from_product(
        [
            ["All", "Thin Disk", "Thick Disk", "Halo"],
            ["Earth", "Super-Earth", "Neptunian", "Sub-Giant", "Giant"],
        ],
    )
)

In [16]:
halo_population_model = PopulationModel(
    fields[fields["Population"] == "Halo"],
    num_embryos=100,
    additional_columns=["Population"],
)

In [ ]:
halo_mocks = [halo_population_model.create_mock_population() for _ in range(100)]
halo_metrics = [
    metrics.calculate_metrics(
        mock,
        number_of_systems=sum(fields["Population"] == "Halo"),
    )
    for mock in halo_mocks
]
halo_table = metrics.calculate_metric_stats(
    halo_metrics, grouping_column="Planet Category"
)

## RECREATING NGPPS II TABLE 6 (INFLUENCE OF EMBRYOS)

In [7]:
results = {}

for num_embryos in [10, 20, 30, 100]:

    print(f"Number of embryos: {num_embryos}, Component: {component}")
    model = PopulationModel(
        fields,
        num_embryos=num_embryos,
        additional_columns=["Population"],
    )

    mock_populations = []
    for i in range(num_mocks):
        mock_populations.append(
            model.create_mock_population(
                add_planet_category=True,
                additional_columns=["Population"],
            )
        )

    del model

    results[num_embryos, component] = metrics.calculate_agg_metrics(
        mock_populations,
        number_of_systems=len(fields),
        population=component,
    )

Number of embryos: 10, Component: Halo
Number of embryos: 10, Component: Thick Disk


: 